# 1. Experimental Setup
- Train/Validation/Test strategy
- Evaluation metrics
- Baseline recap

# 2. Logistic Regression Experiments
    2.1 Default
    2.2 class_weight="balanced"
    2.3 Hyperparameter tuning (C)
    2.4 Threshold tuning

# 3. Random Forest
    3.1 Default
    3.2 Hyperparameter tuning
    3.3 Evaluation

# 4. XGBoost (if included)
    4.1 Default
    4.2 Hyperparameter tuning
    4.3 Evaluation

# 5. Model Comparison

# 6. Champion Model Selection

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import sys
from pathlib import Path

# Get project root (one level up from notebooks/)
project_root = Path().resolve().parents[0]

sys.path.append(str(project_root))

sns.set_style("whitegrid")

RANDOM_STATE = 42


In [ ]:
import pandas as pd
from pathlib import Path

DATA_PATH = Path("../data/processed/telco_churn_semi_raw.csv")
df = pd.read_csv(DATA_PATH)

df.head()
df.info()